# Additional Exercises Solution


### 0. Reading in and pre-processing data

In [6]:
#first, import the necessary modules
import pandas as pd
import numpy as np
#scikit-learn is a huge libaray. We import what we need.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC #Linear Suppot Vector Classifier
from sklearn.naive_bayes import MultinomialNB #Naive Bayes classifier
from sklearn.neighbors import KNeighborsClassifier #nearest neighbors classifier
from sklearn.metrics import accuracy_score #to asses the accuracy of the algorithm
from sklearn.model_selection import cross_val_score #to compute cross validation for assessment purposes

In [38]:
#read our texts and turn them into lists
yelp_path = '../data/yelp.csv' #path may be different on your computer.
yelp = pd.read_csv(yelp_path)
yelp.head(10)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0
5,-yxfBYGB6SEqszmxJxd97A,2007-12-13,m2CKSsepBCoRYWxiRUsxAg,4,"Quiessence is, simply put, beautiful. Full wi...",review,sqYN3lNgvPbPCTRsMFu27g,4,3,1
6,zp713qNhx8d9KCJJnrw1xA,2010-02-12,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doing and drive here. After I...,review,wFweIWhv2fREZV_dYkz_1g,7,7,4
7,hW0Ne_HTHEAgGF1rAdmR-g,2012-07-12,JL7GXJ9u4YMx7Rzs05NfiQ,4,"Luckily, I didn't have to travel far to make m...",review,1ieuYcKS7zeAv_U15AB13A,0,1,0
8,wNUea3IXZWD63bbOQaOH-g,2012-08-17,XtnfnYmnJYi71yIuGsXIUA,4,Definitely come for Happy hour! Prices are ama...,review,Vh_DlizgGhSqQh4qfZ2h6A,0,0,0
9,nMHhuYan8e3cONo3PornJA,2010-08-11,jJAIXA46pU1swYyRCdfXtQ,5,Nobuo shows his unique talents with everything...,review,sUNkXg8-KFtCMQDV6zRzQg,0,1,0


In [15]:
yelp_fivestar_onestar = yelp.loc[(yelp.stars==5) | (yelp.stars==1), :]
yelp_fivestar_onestar['stars'].value_counts()

5    3337
1     749
Name: stars, dtype: int64

### 1. Divide data into training and test sets

First we need to create a training set and a test set. We'll train on the first 500 poems, and test the accuracy on the rest.

In [19]:
#randomize our rows
yelp_fivestar_onestar.sample(3337+749, random_state=0)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
7907,q9WaFYhlOZCrfXJQTG5t_Q,2012-03-08,0OMgIKXCDZPYpiIl03jLPg,5,A real gem!,review,uUoOpU5nXAY2m1qjTynhNQ,0,0,1
5421,ABrSt3fsirLrUYNVrD3fbQ,2011-08-05,clOq6ZQZ86cEVB8NehwEFA,5,Another excellent find in Scottsdale. The ser...,review,pU-c4d5koLUBJuBFKPgNVA,0,0,0
6093,zDfs1Kyk8orFF8bRnSPx-A,2011-10-08,n5vMY3AOigJKP3VyqmPVgA,5,Teresa S. and I were one of the lucky ones to ...,review,0oRtnPHtvZIE1UNxCDDTPw,8,10,6
4744,I6lkxbQq0sSskDlt0jCs9g,2012-12-07,F48k0rnwl5uB9aZ2pKdmsQ,5,Nobody likes going to the auto body shop..peri...,review,DEG-E8qzmZb9dKafebiEfQ,0,0,0
7939,-svamR1_OzUeiZkZf4qotw,2009-09-04,HbZ06PXeK8WXk8i1tmeP-w,5,"This dude claims he ""steals souls for a living...",review,W9n_ZyVCXD0HfPqn6qriOw,0,0,3
8681,NmYTwkqyaCtEg3wifBkxkw,2011-06-29,CTMPP1y-BMZ70UHGj-DAfw,1,"As I promised myself, I'd go back again to try...",review,FgauKBp6w-_8hOY1CRYMoA,0,1,0
9279,tigIXsvKMb4xCiZP56oSPQ,2012-07-24,yVbZt8owdkeBpYh978MmTw,5,We used the taco cart for my 30th birthday thi...,review,H77zOFVd2Jb4pGrNZ5CLTA,0,1,0
1750,NBwPgX7L2D-EZedgo3A-pw,2011-01-15,ruGmXKpLqTj_mTNeiv1rcw,5,Great little hole in the wall with authentic N...,review,e_MwXVPglyJIwE8t-Px-bQ,0,1,0
9200,YKOvlBNkF4KpUP9q7x862w,2011-06-30,N7CUeQkSBc4pdkPGu7aR-g,5,We have been meaning to try The Mission for aw...,review,-uYfeJs1HNUBRDEBYdv7zQ,0,0,0
2356,QxEUzbLfCTXF6gHWJegWgw,2009-03-26,wQUNcLxr-0wVQZzVMJYnKw,5,I am giving Ajo Al's on Shae five stars becaus...,review,rWhikEEkC8iuc5DFlTHXVA,0,1,0


In [48]:
#create two new dataframes
df_train = yelp_fivestar_onestar[:2500]
df_test = yelp_fivestar_onestar[2500:]
print(df_test['stars'].value_counts())
df_train['stars'].value_counts()

5    1278
1     308
Name: stars, dtype: int64


5    2059
1     441
Name: stars, dtype: int64

#### 😁An alternative way to split the data into trainning and test data set. Try it out.

In [49]:
# # define X and y
# text = yelp_fivestar_onestar.text
# stars = yelp_fivestar_onestar.stars
# # split X and y into training and testing sets
# from sklearn.cross_validation import train_test_split
# text_train, text_test, stars_train, stars_test = train_test_split(text, stars, random_state=1)
# #you just put text_train, text_test, stars_train, stars_test in fit_transform(...) transform(...) accordingly

### 2. Supervised Machine Learning Classification

Next we need to create a dtm for each review, and an array containing the classification label for each review (for us, this is called 'label')

In [50]:
#transform the 'text' column into a document term matrix
tfidfvec = TfidfVectorizer(stop_words = 'english', min_df = 1, binary=True)
countvec = CountVectorizer(stop_words = 'english', min_df = 1, binary=True)

training_dtm_tf = countvec.fit_transform(df_train.text)
test_dtm_tf = countvec.transform(df_test.text)

#create an array for stars
training_labels = df_train.stars
test_labels = df_test.stars
test_labels.value_counts()

5    1278
1     308
Name: stars, dtype: int64

### Naive bayes

In [51]:
#define a container for our chosen algorithm, in this case multinomial naive bayes
nb = MultinomialNB()
#fit a model on our training set
nb.fit(training_dtm_tf, training_labels)
#predict the labels on the test set using the trained model
predictions_nb = nb.predict(test_dtm_tf) 
accuracy_score(predictions_nb, test_labels)

0.91046658259773017

In [52]:
#let's see if tf-idf weigting improves the accuracy
training_dtm_tfidf = tfidfvec.fit_transform(df_train.text)
test_dtm_tfidf = tfidfvec.transform(df_test.text)

nb.fit(training_dtm_tfidf, training_labels)
predictions_tfidf = nb.predict(test_dtm_tfidf) 
accuracy_score(predictions_tfidf, test_labels)

0.80643127364438838

### SVC

In [53]:
#let's try a different classifier, LinearSVC
svc = LinearSVC()
svc.fit(training_dtm_tf, training_labels)
predictions_svc = svc.predict(test_dtm_tf) 
accuracy_score(predictions_svc, test_labels)

0.91298865069356872

In [54]:
#let's try a different classifier, LinearSVC using tf-idf
svc = LinearSVC()
svc.fit(training_dtm_tfidf, training_labels)
predictions_svc = svc.predict(test_dtm_tfidf) 
accuracy_score(predictions_svc, test_labels)

0.92875157629255989

### k nearest neighbor

In [55]:
#k nearest neighbor
knn = KNeighborsClassifier()
knn.fit(training_dtm_tf, training_labels)
predictions_knn = knn.predict(test_dtm_tf) 
accuracy_score(predictions_knn, test_labels)

0.8146279949558638

In [56]:
#k nearest neighbor using tf_idf
knn = KNeighborsClassifier()
knn.fit(training_dtm_tfidf, training_labels)
predictions_knn = knn.predict(test_dtm_tfidf) 
accuracy_score(predictions_knn, test_labels)

0.86633039092055486